# Yield Curve Model Testing Notebook

This notebook exercises the `YieldCurve` class by:
1. Building a flat dummy curve for SOFR and USD LIBOR.
2. Computing spot discount factors for various tenors.
3. Computing forward rates for overnight and IBOR indices.

---

## 1) Import dependencies

We start by importing the necessary libraries and the `YieldCurve` class.

In [1]:
import sys, os

# compute the parent of tests → FixedIncomeLib
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

print("Added to sys.path:", repo_root)

Added to sys.path: c:\Users\neels\OneDrive\Desktop\Capstone_Project\FixedIncomeLib


In [2]:
import pandas as pd
import numpy as np
from yield_curve.yield_curve_model import YieldCurve
from data import DataCollection, Data1D, build_yc_data_collection


## 2) Define dummy curve data

Create a small DataFrame with dummy rates for two indices, at 1M, 3M, and 6M tenors.

In [3]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE", "SOFR-FUTURE-3M", "2025-05-05 x 2025-08-05", 97.25],
        ["RFR FUTURE", "SOFR-FUTURE-3M", "2025-08-05 x 2025-11-05", 97.40],
        ["RFR FUTURE", "SOFR-FUTURE-3M", "2025-11-05 x 2026-02-05", 97.55],

        ["RFR SWAP", "USD-SOFR-OIS", "1Y",  0.0450],
        ["RFR SWAP", "USD-SOFR-OIS", "2Y",  0.0425],
        ["RFR SWAP", "USD-SOFR-OIS", "5Y",  0.0370],
        ["RFR SWAP", "USD-SOFR-OIS", "10Y", 0.0350],
    ],
    columns=["DATA TYPE", "DATA CONVENTION", "AXIS", "VALUE"],
)

data_objs, dc = build_yc_data_collection(MARKET_DF)


## 3) Define build methods

Specify how each index should be interpolated; here we use piecewise constant interpolation.

In [4]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT"
}]


## 4) Instantiate the yield curve

Set the valuation date and build the `YieldCurve` object.

In [5]:
value_date = "2025-05-05"
yc = YieldCurve(valueDate=value_date, dataCollection=dc, buildMethodCollection=build_methods)

comp = yc.retrieveComponent("SOFR-1B")
print("Calibration summary:", getattr(comp, "_calibration_summary", {}))

Calibration summary: {'rmse': 0.03168033568768038, 'max_abs': 0.044712553290740115, 'n_residuals': 7, 'n_parameters': 26}


In [6]:
yc.components.keys()

dict_keys(['SOFR-1B'])

## 5) Compute and display spot discount factors

We pick several (index, date) pairs to show discount factors.

In [7]:
print("--- Spot Discount Factors ---")
tests = [
    ("SOFR-1B",          "2025-07-26"),
    ("SOFR-1B",          "2025-09-26"),
]
for idx, dt in tests:
    df_val = yc.discountFactor(idx, dt)
    print(f"{idx:>18} @ {dt}: DF = {df_val:.6f}")


--- Spot Discount Factors ---
           SOFR-1B @ 2025-07-26: DF = 0.993673
           SOFR-1B @ 2025-09-26: DF = 0.993189


## 6) Compute and display forward rates

Finally, we compute 1M and 3M overnight forwards, and a 3M LIBOR forward:

In [8]:
print("\n--- Forward Rates ---")
fwd_1m    = yc.forward("SOFR-1B",          value_date, "1M")
fwd_3m    = yc.forward("SOFR-1B",          value_date, "3M")

print(f"SOFR 1M forward:  {fwd_1m:.6%}")
print(f"SOFR 3M forward:  {fwd_3m:.6%}")



--- Forward Rates ---
SOFR 1M forward:  2.733246%
SOFR 3M forward:  2.769463%


In [9]:
starting_date = "2026-09-12"
tenor = "3M"

yc.forward("SOFR-1B", starting_date, tenor)

np.float64(0.0748288283294089)